参考：[【連載】LangChainの公式チュートリアルを1個ずつ地味に、地道にコツコツと【Basic編#3】](https://zenn.dev/chips0711/articles/25c11940a999a1)

### LangChainのドキュメントローダー([公式参考](https://python.langchain.com/docs/how_to/#document-loaders))
様々なデータ形式からテキストデータを読み込み、LangChainで処理できるDocumentオブジェクトに変換する機能  
Documentオブジェクトは、テキストデータとそのメタデータを格納するための標準的な形式  
以下のようなファイルからの読み込みに対応している他、自作のドキュメントローダーを作成することも可能  
 - CSVローダー
 - ディレクトリローダー
 - HTMLローダー
 - JSONローダー
 - Markdownローダー
 - Markdownを要素ごとに分割して読み込む
 - Officeファイルローダー
 - PDFローダー

### TextSplitter([公式参考](https://python.langchain.com/docs/how_to/#text-splitters))
 - テキストを意味のある単位に分割するためのインターフェース
 - コンテキストウィンドウのサイズ制限に対応したり、各チャンクを個別に処理したりすることができる  
 
※参考サイトのコードはライブラリバージョンの関係でそのまま動作しなかったため、  
　公式のサンプルを元に修正している

##### RecursiveCharacterTextSplitterの例
 - chunk_overlapパラメータで、チャンク間の重複する文字数を指定できる

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# テキスト分割器を初期化
long_text = "これは、RecursiveCharacterTextSplitterの使用例です。このテキストは、チャンクサイズに応じて分割されます。分割されたチャンクは、それぞれ独立して処理されます。"
text_splitter = RecursiveCharacterTextSplitter(chunk_size=20, chunk_overlap=5)

# テキストを分割
texts = text_splitter.create_documents([long_text])

# 分割されたテキストを表示
for i, text in enumerate(texts):
    print(f"チャンク {i+1}: {text.page_content}")

チャンク 1: これは、RecursiveCharact
チャンク 2: aracterTextSplitterの
チャンク 3: tterの使用例です。このテキストは、チ
チャンク 4: ストは、チャンクサイズに応じて分割されま
チャンク 5: 分割されます。分割されたチャンクは、それ
チャンク 6: クは、それぞれ独立して処理されます。


##### MarkdownHeaderTextSplitterの例
 - headers_to_split_onパラメータで見出しレベルを指定することができる

In [5]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

markdown_text = """# 見出し1
これは、見出し1のセクションです。

## 見出し1.1
これは、見出し1.1のセクションです。

## 見出し1.2
これは、見出し1.2のセクションです。

# 見出し2
これは、見出し2のセクションです。
"""

# MarkdownHeaderTextSplitterを初期化
# headers_to_split_on = ["# ", "## "]
headers_to_split_on = [("#", "Header 1"), ("##", "Header 2")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# テキストを分割
markdown_docs = markdown_splitter.split_text(markdown_text)

# 分割されたドキュメントを表示
for i, doc in enumerate(markdown_docs):
    print(f"ドキュメント {i+1}:\n{doc.page_content}")

ドキュメント 1:
これは、見出し1のセクションです。
ドキュメント 2:
これは、見出し1.1のセクションです。
ドキュメント 3:
これは、見出し1.2のセクションです。
ドキュメント 4:
これは、見出し2のセクションです。


##### HTMLHeaderTextSplitterの例
 - headers_to_split_onパラメータで見出しレベルを指定

In [11]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_text = """<h1>見出し1</h1>
<p>これは、見出し1のセクションです。</p>

<h2>見出し1.1</h2>
<p>これは、見出し1.1のセクションです。</p>

<h3>見出し1.1.1</h3>
<p>これは、見出し1.1.1のセクションです。</p>

<h1>見出し2</h1>
<p>これは、見出し2のセクションです。</p>
"""

# HTMLHeaderTextSplitterを初期化
headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2"), ("h3", "Header 3")]
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# テキストを分割
html_docs = html_splitter.split_text(html_text)

# 分割されたドキュメントを表示
for i, doc in enumerate(html_docs):
    print(f"ドキュメント {i+1}:\n{doc.page_content}")

ドキュメント 1:
これは、見出し1のセクションです。
ドキュメント 2:
これは、見出し1.1のセクションです。
ドキュメント 3:
これは、見出し1.1.1のセクションです。
ドキュメント 4:
これは、見出し2のセクションです。


##### CodeSplitterの例
 - プログラミング言語に合わせて適切な分割を行える

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language

python_code = """
def my_function(a, b):
    # これは関数です。
    return a + b

class MyClass:
    # これはクラスです。

    def __init__(self, x):
        self.x = x
"""

# CodeSplitterを初期化
code_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
    )

# コードを分割
code_docs = code_splitter.create_documents([python_code])

# 分割されたドキュメントを表示
for i, doc in enumerate(code_docs):
    print(f"ドキュメント {i+1}:\n{doc.page_content}")

ドキュメント 1:
def my_function(a, b):
    # これは関数です。
ドキュメント 2:
return a + b
ドキュメント 3:
class MyClass:
    # これはクラスです。
ドキュメント 4:
def __init__(self, x):
        self.x = x


##### SemanticTextSplitterの例
 - 意味的に関連する文章をまとめてチャンク化することで、より自然な分割を行える  
　→　分割できなかった。長い文章にするとAPI_KEYの認証エラーが出る

In [20]:
import os

from dotenv import load_dotenv, find_dotenv

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

load_dotenv(find_dotenv())


# SemanticTextSplitterを初期化
text_splitter = SemanticChunker(OpenAIEmbeddings(api_key=os.environ["AZURE_OPENAI_API_KEY"]))

# テキストを分割
text = "これは、SemanticTextSplitterの使用例です。このテキストは、意味的なまとまりに応じて分割されます。意味的な分割により、各チャンクがより自然な単位になります。"
texts = text_splitter.create_documents([text])

# 分割されたテキストを表示
for i, text in enumerate(texts):
    print(f"チャンク {i+1}: {text.page_content}")

チャンク 1: これは、SemanticTextSplitterの使用例です。このテキストは、意味的なまとまりに応じて分割されます。意味的な分割により、各チャンクがより自然な単位になります。


### Text Embedding(テキストの埋め込み)


### 埋め込みを活用した検索システムの構築

##### ベクトルストア

埋め込みベクトルを格納・検索するためのデータベース  
LangChainでは、Chroma、FAISS、Pineconeなど、様々なベクトルストアをサポートしている  
以下のような共通機能がある

 - 埋め込みベクトルの追加:
   - add_textsメソッドやadd_documentsメソッドを使って、埋め込みベクトルを追加できる
 - 類似度検索:
   - similarity_searchメソッドやsimilarity_search_with_scoreメソッドを使って、クエリに類似したドキュメントを検索できる
 - その他の検索方法:
   - ベクトルストアによっては、最大マージン関連検索（MMR）やスコア閾値検索などの検索方法も提供している

##### FAISS (Facebook AI Similarity Search)[[公式参考](https://python.langchain.com/docs/how_to/vectorstores/)]
 - 高速な検索性能に特化したベクトルストア
 - 大規模なデータセットに対しても高速な検索を実現できる
 - C++で実装されており、Pythonからも利用可能  
　→　こちらも動作せず(API_KEY関連のエラー)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

load_dotenv(find_dotenv())

# AzureOpenAIEmbeddingsインスタンスを作成します
embeddings = AzureOpenAIEmbeddings(
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

# 埋め込み対象のドキュメントを作成
documents = [
    Document(page_content="犬は忠実な友達です。", metadata={"source": "ペット百科事典"}),
    Document(page_content="猫は独立心が強いです。", metadata={"source": "猫の飼い方ガイド"}),
]

# FAISSベクトルストアを初期化
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings(api_key=os.environ["AZURE_OPENAI_API_KEY"]))

# 検索クエリを埋め込み
query_embedding = embeddings.embed_query("猫")

# MMRを使って類似するドキュメントを検索
results = vectorstore.max_marginal_relevance_search(
    query_embedding, k=2, fetch_k=4
)

for result in results:
    print(f"コンテンツ: {result.page_content}, メタデータ: {result.metadata}")